# This app extracts cryptocurrencies' data from Coingecko's API

## APIs used:

    - Get/coins/markets to get a list of all supported coins, ath, price, volume and other info
    - Get/coins/{id}/history to get community and developer data for specific dates
    

## After extracting the data, it gets saved as a CSV
    - Only run app once, or saved csv will get overwritten
    - If running more than once, change name for csv
    

## Further APIs to use:
    - https://api.coingecko.com/api/v3/coins/categories


In [2]:
import requests
import pandas as pd
import time
import datetime

In [2]:
# Create an empty list to hold the coins information
crypto_list = []

In [3]:
# Find an API that has information on coin and make several requests

# Goal is 2,500 entries, each page has 300. Make a loop to get 10 pages

for i in range(1,10,1):
    page_number = i
    base_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page={page_number}&sparkline=false"
    
    # Test out the URL
    #print(base_url)

    # Request json file from the url
    response_json = requests.get(base_url).json()

    # append response to empty list
    crypto_list.append(response_json)

In [4]:
# Make each page entry into a datafrme and concat dataframes together

# Set up the a value for the previous dataframe
previous_df = pd.DataFrame(crypto_list[0])
# Loop through the entries in the list
for i in range(1, len(crypto_list)):
    next_df = pd.DataFrame(crypto_list[i])
    complete_df = pd.concat([previous_df, next_df])
    previous_df = complete_df



In [5]:
# reset the index
clean_complete_df = complete_df.reset_index(drop=True)
#clean_complete_df

In [6]:
# See the name of the columns
#clean_complete_df.columns

In [59]:
# Only get specific columns
#crypto_df = clean_complete_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]
#crypto_df

In [8]:
# Only run this cell if the data hasn't been renamed and cleaned
if crypto_df.columns[3] == "market_cap":
    # Clean data by dropping columns, ONLY RUN IT ONCE, since columns will change name
    #crypto_df = crypto_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]
    crypto_df['market_cap'] = crypto_df['market_cap']/1000000000

    # Rename columns to see easier
    crypto_df.rename(columns={'market_cap':'market_cap_billion(2021)',
                              'current_price' : 'price_04-22-21'

                             }, inplace=True)


<ipython-input-8-3d38269c4a46>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crypto_df['market_cap'] = crypto_df['market_cap']/1000000000
/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
# Verify data drame
#crypto_df.head()

In [10]:
# Save data as csv
# crypto_df.to_csv("data/cryptocurrencies_data_2021.csv")

In [69]:
# Get each coin's price, and socialdata on 01-04-2018 
# Make empty list to hold the value of coins

coin_data_2018 = []

target_date = "01-04-2018"

In [12]:
# Set up timer to slow down requests speed
def timer_milliseconds(milliseconds = 3):
    for i in range(0, milliseconds):
        print(f"Counting... {i +1}")
        time.sleep(1)
        

####################################################################################
## Further work: make function to update date list automatically
####################################################################################
### 1. Get current date
### 2. Get most recent date from date list through formated_date[0]
### 3. Find the number of days between 1 and 2
### 4. make new dates list
### 5. insert new list at the top of old list
### 6. Rewrite old list

In [5]:
# Make list of dates to make a request for each date

# Create a date object called base
base = datetime.datetime.today()

# set up how many days you want the dates of
numdays = 1460

# Use a list comprehension to create a list of dates

date_list = [base - datetime.timedelta(days=x) for x in range(numdays)] 


# Format the list of dates mm-dd- based in the format required for the api endpoint
missing_dates = [base.strftime('%d-%m-20%y') for base in date_list]


len(missing_dates)

datetime.timedelta(days=4)

## This is the function that requests data for each cryptocurrency from each date

In [118]:
# This function gets data on a token for the last 4 years. It takes 4 parameters: 


# 1. Where in the date index we want to start; most recent date is july 29, 2021 which is the default

# 2. Last index from date to get; date list goes back 1460 days or 4 years which is the default

# 3. What token you would like to search for; default is bitcoin

# 4. What list to append the results to; default is the bitcoin list, need to make empty list before running


def get_data_for_missing_dates(index_start = 0, last_index_wanted = len(missing_dates), token = 'bitcoin', append_to = bitcoin_list):

    for x in range(index_start, last_index_wanted):


        date = {'date': missing_dates[x]}
        print("now checking:", missing_dates[x])
        print(x)
        url = f"https://api.coingecko.com/api/v3/coins/{token}/history?date={missing_dates[x]}localization=False"

        try:
            
#           If the request is succesful, combine the date dictionary with the response json
    
            response_json = requests.get(url).json()
            date.update(response_json)
            
#           Append the combined dictionary to our desired list

            append_to.append(date)
    
            print('success')
        except:
            print(f"Looks like you have been timed out, continue at {x} for {missing_dates[x]}")
            
            break

In [121]:
# keep collecting data until entries is same number as date list

while len(bitcoin_list) < len(formated_date):
    get_data_last_four_years(index_start = len(bitcoin_list))

now checking: 27-01-2020
549
success
now checking: 26-01-2020
550
success
now checking: 25-01-2020
551
success
now checking: 24-01-2020
552
success
now checking: 23-01-2020
553
success
now checking: 22-01-2020
554
success
now checking: 21-01-2020
555
success
now checking: 20-01-2020
556
success
now checking: 19-01-2020
557
success
now checking: 18-01-2020
558
success
now checking: 17-01-2020
559
success
now checking: 16-01-2020
560
success
now checking: 15-01-2020
561
success
now checking: 14-01-2020
562
success
now checking: 13-01-2020
563
success
now checking: 12-01-2020
564
success
now checking: 11-01-2020
565
success
now checking: 10-01-2020
566
success
now checking: 09-01-2020
567
success
now checking: 08-01-2020
568
success
now checking: 07-01-2020
569
success
now checking: 06-01-2020
570
success
now checking: 05-01-2020
571
success
now checking: 04-01-2020
572
success
now checking: 03-01-2020
573
success
now checking: 02-01-2020
574
success
now checking: 01-01-2020
575
success
n

Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, continue at 609 for 28-11-2019
now checking: 28-11-2019
609
Looks like you have been timed out, cont

Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, cont

669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, 

Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, continue at 669 for 29-09-2019
now checking: 29-09-2019
669
Looks like you have been timed out, cont

...

Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have been timed out, continue at 1389 for 09-10-2017
now checking: 09-10-2017
1389
Looks like you have 

success
now checking: 16-09-2017
1412
Looks like you have been timed out, continue at 1412 for 16-09-2017
now checking: 16-09-2017
1412
success
now checking: 15-09-2017
1413
success
now checking: 14-09-2017
1414
Looks like you have been timed out, continue at 1414 for 14-09-2017
now checking: 14-09-2017
1414
Looks like you have been timed out, continue at 1414 for 14-09-2017
now checking: 14-09-2017
1414
Looks like you have been timed out, continue at 1414 for 14-09-2017
now checking: 14-09-2017
1414
Looks like you have been timed out, continue at 1414 for 14-09-2017
now checking: 14-09-2017
1414
Looks like you have been timed out, continue at 1414 for 14-09-2017
now checking: 14-09-2017
1414
Looks like you have been timed out, continue at 1414 for 14-09-2017
now checking: 14-09-2017
1414
success
now checking: 13-09-2017
1415
success
now checking: 12-09-2017
1416
Looks like you have been timed out, continue at 1416 for 12-09-2017
now checking: 12-09-2017
1416
Looks like you have been t

Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
Looks like you have been timed out, continue at 1436 for 23-08-2017
now checking: 23-08-2017
1436
success
now checking: 22-08-2017
1437
Looks like you have been timed out, continue at 1437 for 22-08-2017
now checking: 22-08-2017
1437
success
now checking: 21-08-2017
1438
Looks like you have been timed out, contin

Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have 

Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have been timed out, continue at 1449 for 10-08-2017
now checking: 10-08-2017
1449
Looks like you have 

In [10]:
# Verify list
len(bitcoin_list)

NameError: name 'bitcoin_list' is not defined

In [126]:
# make the list into a dataframe to save is as a csv

bitcoin_df = pd.DataFrame(bitcoin_list)
bitcoin_df.to_csv('../static/data/bitcoin_four_years_data.csv')
bitcoin_df.head()

,date,id,symbol,name,localization,image,market_data,community_data,developer_data,public_interest_stats
0,29-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 147042.59739150605, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 29494, 'stars': 55888, 'subscribers'...","{'alexa_rank': 6058, 'bing_matches': None}"
1,28-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 143536.14423828595, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 29485, 'stars': 55854, 'subscribers'...","{'alexa_rank': 6058, 'bing_matches': None}"
2,27-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 136937.45448243525, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 29476, 'stars': 55832, 'subscribers'...","{'alexa_rank': 6058, 'bing_matches': None}"
3,26-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 130231.05500148985, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 6058, 'bing_matches': None}"
4,25-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 125673.39527087075, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 6058, 'bing_matches': None}"


In [ ]:
# ethereum_list = []

In [9]:
# Get data for Ethereum

while len(ethereum_list) < len(formated_date):
    get_data_last_four_years(index_start = len(ethereum_list), last_index_wanted = len(formated_date), token = 'ethereum', append_to = ethereum_list)

NameError: name 'ethereum_list' is not defined

In [8]:
ethereum_list

NameError: name 'ethereum_list' is not defined

In [263]:
ethereum_df = pd.DataFrame(ethereum_list)
ethereum_df.to_csv('../static/data/ethereum_four_years_data.csv')
ethereum_df.head()

,date,id,symbol,name,localization,image,market_data,community_data,developer_data,public_interest_stats
0,29-07-2021,ethereum,eth,Ethereum,"{'en': 'Ethereum', 'de': 'Ethereum', 'es': 'Et...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 8447.219125885169, '...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 3522, 'bing_matches': None}"
1,28-07-2021,ethereum,eth,Ethereum,"{'en': 'Ethereum', 'de': 'Ethereum', 'es': 'Et...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 8421.103521907726, '...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 11594, 'stars': 31582, 'subscribers'...","{'alexa_rank': 3522, 'bing_matches': None}"
2,27-07-2021,ethereum,eth,Ethereum,"{'en': 'Ethereum', 'de': 'Ethereum', 'es': 'Et...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 8191.626910078455, '...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 3522, 'bing_matches': None}"
3,26-07-2021,ethereum,eth,Ethereum,"{'en': 'Ethereum', 'de': 'Ethereum', 'es': 'Et...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 8115.535339645566, '...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 11583, 'stars': 31545, 'subscribers'...","{'alexa_rank': 3522, 'bing_matches': None}"
4,25-07-2021,ethereum,eth,Ethereum,"{'en': 'Ethereum', 'de': 'Ethereum', 'es': 'Et...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 8020.900219153302, '...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 3522, 'bing_matches': None}"


# This function unpacks the data so that we can make a table only for community data

In [266]:
# Make function to unpack columns from dataframe and save them as csvs
def unpack_data(symbol = 'btc', column = 'community_data', list_to_unpack = bitcoin_list):
    
    empty_list = []
    for i in range (0, len(list_to_unpack)):
        coin = {'id': list_to_unpack[i]['id']}
        date = {'date':list_to_unpack[i]['date']}
        data = list_to_unpack[i][f'{column}']
        date.update(coin)
        date.update(data)
        empty_list.append(date)
    # Save data as csv
    
    pd.DataFrame(empty_list).to_csv(f'../static/data/{symbol}_{column}.csv')
    
    

# Since the market data json structure is different, We had to make another function to unpack that data

In [268]:
# Make list to unpack market data since its structure is different

def unpack_market_data(symbol ='btc', column = 'market_data', list_to_unpack = bitcoin_list):
    
    empty_list = []
    
    for i in range (0, len(list_to_unpack)):
        coin = {'id': list_to_unpack[i]['id']}
        date = {'date':list_to_unpack[i]['date']}
        data = {'price' : list_to_unpack[i][f'{column}']['current_price']['usd'],
                'market_cap': list_to_unpack[i][f'{column}']['market_cap']['usd'],
                'total_volume': list_to_unpack[i][f'{column}']['total_volume']['usd']
               }
        date.update(coin)
        date.update(data)
        empty_list.append(date)
        
    # Save data as csv
    
    pd.DataFrame(empty_list).to_csv(f'../static/data/{symbol}_{column}.csv')

In [269]:
# function to unpack all data and save it as csvs, it takes a token name and a list with data

def unpack_all_columns_to_csv(token, list_to_unpack):
    
    # Make empty lists
#     token_developer_data = []
#     token_market_data = []
#     token_community_data = []
#     token_public_interest_stats =[]

    unpack_data(token, 'community_data', list_to_unpack)
    unpack_data(token,'developer_data', list_to_unpack)
    unpack_market_data(token, 'market_data', list_to_unpack)
    unpack_data(token, 'public_interest_stats', list_to_unpack)



In [270]:
unpack_all_columns_to_csv('eth', ethereum_list)

In [128]:
bitcoin_df.to_csv('../static/data/bitcoin_four_years_data.csv')

In [218]:
pd.read_csv('../static/data/bitcoin_four_years_data.csv')

,Unnamed: 0,date,id,symbol,name,localization,image,market_data,community_data,developer_data,public_interest_stats
0,0,29-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 147042.59739150605, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 29494, 'stars': 55888, 'subscribers'...","{'alexa_rank': 6058, 'bing_matches': None}"
1,1,28-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 143536.14423828595, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 29485, 'stars': 55854, 'subscribers'...","{'alexa_rank': 6058, 'bing_matches': None}"
2,2,27-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 136937.45448243525, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 29476, 'stars': 55832, 'subscribers'...","{'alexa_rank': 6058, 'bing_matches': None}"
3,3,26-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 130231.05500148985, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 6058, 'bing_matches': None}"
4,4,25-07-2021,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 125673.39527087075, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': None, 'stars': None, 'subscribers': ...","{'alexa_rank': 6058, 'bing_matches': None}"
...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,04-08-2017,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 10453.020119698383, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 9412, 'stars': 15043, 'subscribers':...","{'alexa_rank': 5570, 'bing_matches': None}"
1456,1456,03-08-2017,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 10201.542046459217, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 9388, 'stars': 15002, 'subscribers':...","{'alexa_rank': 5622, 'bing_matches': None}"
1457,1457,02-08-2017,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 9908.610139104734, '...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 9361, 'stars': 14949, 'subscribers':...","{'alexa_rank': 5654, 'bing_matches': None}"
1458,1458,01-08-2017,bitcoin,btc,Bitcoin,"{'en': 'Bitcoin', 'de': 'Bitcoin', 'es': 'Bitc...",{'thumb': 'https://assets.coingecko.com/coins/...,"{'current_price': {'aed': 10056.071972643549, ...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 9333, 'stars': 14913, 'subscribers':...","{'alexa_rank': 5694, 'bing_matches': None}"


In [130]:
# Read what are the names of the columns to make a table is posgres
len(bitcoin_df.columns)

10

In [67]:
# Make function to perform request based on specific data, use dataframe above's id to search that specific coin
def obtain_metrics_by_date (target_df = crypto_df, target_list = coin_data_2018, date='01-04-2018'):
    # Set up error counter to break out if there are too many errors
    key_error_counter = 0
    timeout_error_counter = 0
    #iterate through dataframe getting index and row
    for index, row in target_df.iterrows():
        
        #get coin name through dictionary syntax
        target_coin = row['id']
        
        # print the current row's coin
        print(f"Processing {target_coin}")
        
        # If there are less than the number of errors allowed, try doing a request
        
        if (key_error_counter < 150) and (timeout_error_counter < 2):
            try:
                
                # Build url
                price_url = f"https://api.coingecko.com/api/v3/coins/{target_coin}/history?date={date}localization=False"
                
                # Print message to keep user informed; url being requested and index
                
                print(f"Requesting from URL: {price_url}")
                print(f"At index: {index}")
                
                # save json respons
                response_json = requests.get(price_url).json()
                
                # traverse json getting specific data
                coin_id = response_json['id']
                coin_price = response_json['market_data']['current_price']['usd']
                market_cap = response_json['market_data']['market_cap']['usd']
                coin_reddit_subscribers = response_json['community_data']['reddit_subscribers']
                coin_twitter_followers = response_json['community_data']['twitter_followers']
                coin_alexa_rank = response_json['public_interest_stats']['alexa_rank']
                
                # Add data to empty list in dictionary style
                target_list.append({
                    "id" : coin_id,
                    f"price {date}" : coin_price,
                    f"market_cap" : market_cap,
                    f"reddit_subscribers {date}" : coin_reddit_subscribers,
                    f"twitter_followers {date}" : coin_twitter_followers,
                    f"alexa_rank {date}" : coin_alexa_rank
                       })
                
            # The first error is a no market_data for that coin
            except :
                try:
                    # The first error will be a key error, since young coins dont have 'market_data'
                    # add one to the key_error counter
                    key_error_counter+=1
                    
                    #print message
                    print("Key error. Coin too young to find all data")   
                    print(f"Retrying {target_coin} to make sure we are not timedout")
                    print(f"Retrying URL: {price_url}")
                    
                    # make a request again, in case we got timedout
                    response_json = requests.get(price_url).json()
                    #still add the coin to the list
                    target_list.append({'id': coin_id})
                 
                # the next error would be a timedout error
                except :
                    print('Ruh Roh! Looks like you have been timed out')
                    print(f"The last coin to be pulled was {target_coin} at index {index}")
                    
                    # If we got timed out, we want to exit the function
                    timeout_error_counter += 1
                    timer_milliseconds(30)
                    #break
        # print message for too many requests errors
        else:
            print(f"You have reached {key_error_counter} key errors and {timeout_error_counter} timeouts.")
            print(f"Last data pulled at position index:{index}, id: {target_coin}")
            break
    print("Done with Data Retieval")
    print(f"We found a total of {len(target_list)} entries")
  



In [14]:
obtain_metrics_by_date()

Processing bitcoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-04-2018localization=False
At index: 0
Processing ethereum
Requesting from URL: https://api.coingecko.com/api/v3/coins/ethereum/history?date=01-04-2018localization=False
At index: 1
Processing binancecoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/binancecoin/history?date=01-04-2018localization=False
At index: 2
Processing ripple
Requesting from URL: https://api.coingecko.com/api/v3/coins/ripple/history?date=01-04-2018localization=False
At index: 3
Processing tether
Requesting from URL: https://api.coingecko.com/api/v3/coins/tether/history?date=01-04-2018localization=False
At index: 4
Processing cardano
Requesting from URL: https://api.coingecko.com/api/v3/coins/cardano/history?date=01-04-2018localization=False
At index: 5
Processing dogecoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/dogecoin/history?date=01-04-2018localization=False
At index: 6
Proce

Key error. Coin too young to find all data
Retrying okb to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/okb/history?date=01-04-2018localization=False
Processing compound-usd-coin
Requesting from URL: https://api.coingecko.com/api/v3/coins/compound-usd-coin/history?date=01-04-2018localization=False
At index: 35
Key error. Coin too young to find all data
Retrying compound-usd-coin to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/compound-usd-coin/history?date=01-04-2018localization=False
Processing ethereum-classic
Requesting from URL: https://api.coingecko.com/api/v3/coins/ethereum-classic/history?date=01-04-2018localization=False
At index: 36
Processing compound-ether
Requesting from URL: https://api.coingecko.com/api/v3/coins/compound-ether/history?date=01-04-2018localization=False
At index: 37
Key error. Coin too young to find all data
Retrying compound-ether to make sure we are not timedout
Retrying URL: http

In [15]:
len(coin_data_2018)

45

In [25]:
# If we got timed out, make a DF starting from the last token proccesed to run the function again
# last_coin_proccesed = len(coin_data_2018)
# left_over_df = crypto_df.iloc[last_coin_proccesed:]
# left_over_df



In [32]:
# Keep obtaining data until our df is at least 500 short from the original list
def persistent_scrapping(list_to_append_entries = coin_data_2018, number_entries_desired = len(crypto_df) -500, date = '01-04-2018'):
    
    # Set up a leftover dataframe based on the original dataframe we were trying to use for scrapping
    last_coin_proccesed = len(list_to_append_entries)
    left_over_df = crypto_df.iloc[last_coin_proccesed:]
    
    # Keep obtaining data until our desired threshold 
    while len(list_to_append_entries) < number_entries_desired:
        
        # Call function above
        obtain_metrics_by_date(left_over_df, list_to_append_entries, date)
        # Get the index for the last coin pricesses
        last_coin_proccesed = len(list_to_append_entries)
        # Update the leftovers data frame starting from the last coin_processed
        left_over_df = crypto_df.iloc[last_coin_proccesed:]



In [86]:
persistent_scrapping(coin_data_2018, 300)

Processing titanswap
Requesting from URL: https://api.coingecko.com/api/v3/coins/titanswap/history?date=01-04-2018localization=False
At index: 219
Key error. Coin too young to find all data
Retrying titanswap to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/titanswap/history?date=01-04-2018localization=False
Processing hathor
Requesting from URL: https://api.coingecko.com/api/v3/coins/hathor/history?date=01-04-2018localization=False
At index: 220
Key error. Coin too young to find all data
Retrying hathor to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/hathor/history?date=01-04-2018localization=False
Processing badger-dao
Requesting from URL: https://api.coingecko.com/api/v3/coins/badger-dao/history?date=01-04-2018localization=False
At index: 221
Key error. Coin too young to find all data
Retrying badger-dao to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/badger-dao/history?d

Key error. Coin too young to find all data
Retrying keep-network to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/keep-network/history?date=01-04-2018localization=False
Processing compound-0x
Requesting from URL: https://api.coingecko.com/api/v3/coins/compound-0x/history?date=01-04-2018localization=False
At index: 247
Key error. Coin too young to find all data
Retrying compound-0x to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/compound-0x/history?date=01-04-2018localization=False
Processing olympus
Requesting from URL: https://api.coingecko.com/api/v3/coins/olympus/history?date=01-04-2018localization=False
At index: 248
Key error. Coin too young to find all data
Retrying olympus to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/olympus/history?date=01-04-2018localization=False
Processing monacoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/monacoin/history?dat

Key error. Coin too young to find all data
Retrying fx-coin to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/fx-coin/history?date=01-04-2018localization=False
Ruh Roh! Looks like you have been timed out
The last coin to be pulled was fx-coin at index 275
Counting... 1
Counting... 2
Counting... 3
Counting... 4
Counting... 5
Counting... 6
Counting... 7
Counting... 8
Counting... 9
Counting... 10
Counting... 11
Counting... 12
Counting... 13
Counting... 14
Counting... 15
Counting... 16
Counting... 17
Counting... 18
Counting... 19
Counting... 20
Counting... 21
Counting... 22
Counting... 23
Counting... 24
Counting... 25
Counting... 26
Counting... 27
Counting... 28
Counting... 29
Counting... 30
Processing ethernity-chain
Requesting from URL: https://api.coingecko.com/api/v3/coins/ethernity-chain/history?date=01-04-2018localization=False
At index: 276
Key error. Coin too young to find all data
Retrying ethernity-chain to make sure we are not timedout
Retryin

Key error. Coin too young to find all data
Retrying render-token to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/render-token/history?date=01-04-2018localization=False
Processing sharering
Requesting from URL: https://api.coingecko.com/api/v3/coins/sharering/history?date=01-04-2018localization=False
At index: 295
Key error. Coin too young to find all data
Retrying sharering to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/sharering/history?date=01-04-2018localization=False
Processing compound-basic-attention-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/compound-basic-attention-token/history?date=01-04-2018localization=False
At index: 296
Key error. Coin too young to find all data
Retrying compound-basic-attention-token to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/compound-basic-attention-token/history?date=01-04-2018localization=False
Processing deriv

Key error. Coin too young to find all data
Retrying auction to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/auction/history?date=01-04-2018localization=False
Processing divi
Requesting from URL: https://api.coingecko.com/api/v3/coins/divi/history?date=01-04-2018localization=False
At index: 321
Key error. Coin too young to find all data
Retrying divi to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/divi/history?date=01-04-2018localization=False
Processing zcoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/zcoin/history?date=01-04-2018localization=False
At index: 322
Processing zenon
Requesting from URL: https://api.coingecko.com/api/v3/coins/zenon/history?date=01-04-2018localization=False
At index: 323
Key error. Coin too young to find all data
Retrying zenon to make sure we are not timedout
Retrying URL: https://api.coingecko.com/api/v3/coins/zenon/history?date=01-04-2018localization=False
Process

In [88]:
pd.DataFrame(coin_data_2018)

,id,price 01-04-2018,market_cap,reddit_subscribers 01-04-2018,twitter_followers 01-04-2018,alexa_rank 01-04-2018
0,bitcoin,6975.275314,1.182341e+11,785619.0,819312.0,5574.0
1,ethereum,395.796381,3.900087e+10,348116.0,382159.0,8204.0
2,binancecoin,11.115511,1.100591e+09,40605.0,176.0,230.0
3,ripple,0.511990,2.001600e+10,184063.0,843082.0,5224.0
4,tether,1.000690,2.288719e+09,NaN,23302.0,34929.0
...,...,...,...,...,...,...
324,marlin,NaN,NaN,NaN,NaN,NaN
325,nominex,NaN,NaN,NaN,NaN,NaN
326,mx-token,NaN,NaN,NaN,NaN,NaN
327,sparkpoint,NaN,NaN,NaN,NaN,NaN


In [93]:
coin_data_2018

[{'id': 'bitcoin',
  'price 01-04-2018': 6975.27531402076,
  'market_cap': 118234055446.5432,
  'reddit_subscribers 01-04-2018': 785619,
  'twitter_followers 01-04-2018': 819312,
  'alexa_rank 01-04-2018': 5574},
 {'id': 'ethereum',
  'price 01-04-2018': 395.79638095782116,
  'market_cap': 39000868214.27854,
  'reddit_subscribers 01-04-2018': 348116,
  'twitter_followers 01-04-2018': 382159,
  'alexa_rank 01-04-2018': 8204},
 {'id': 'binancecoin',
  'price 01-04-2018': 11.115510878254739,
  'market_cap': 1100591194.0995147,
  'reddit_subscribers 01-04-2018': 40605,
  'twitter_followers 01-04-2018': 176,
  'alexa_rank 01-04-2018': 230},
 {'id': 'ripple',
  'price 01-04-2018': 0.51199,
  'market_cap': 20016003613.76977,
  'reddit_subscribers 01-04-2018': 184063,
  'twitter_followers 01-04-2018': 843082,
  'alexa_rank 01-04-2018': 5224},
 {'id': 'tether',
  'price 01-04-2018': 1.00069,
  'market_cap': 2288718941.16166,
  'reddit_subscribers 01-04-2018': None,
  'twitter_followers 01-04-20

In [95]:
# Data cleanup
coins_2018_df = pd.DataFrame(coin_data_2018)
coins_2018_df['market_cap'] = coins_2018_df['market_cap'] / 1000000
coins_2018_df.rename(columns={
    'market_cap' : 'market_cap_2018(millions)'
},inplace=True)
coins_2018_df


,id,price 01-04-2018,market_cap_2018(millions),reddit_subscribers 01-04-2018,twitter_followers 01-04-2018,alexa_rank 01-04-2018
0,bitcoin,6975.275314,118234.055447,785619.0,819312.0,5574.0
1,ethereum,395.796381,39000.868214,348116.0,382159.0,8204.0
2,binancecoin,11.115511,1100.591194,40605.0,176.0,230.0
3,ripple,0.511990,20016.003614,184063.0,843082.0,5224.0
4,tether,1.000690,2288.718941,NaN,23302.0,34929.0
...,...,...,...,...,...,...
324,marlin,NaN,NaN,NaN,NaN,NaN
325,nominex,NaN,NaN,NaN,NaN,NaN
326,mx-token,NaN,NaN,NaN,NaN,NaN
327,sparkpoint,NaN,NaN,NaN,NaN,NaN


In [96]:
coins_2018_df.nunique()

id                               328
price 01-04-2018                 104
market_cap_2018(millions)        100
reddit_subscribers 01-04-2018     92
twitter_followers 01-04-2018     103
alexa_rank 01-04-2018            100
dtype: int64

In [110]:
coins_2018_df.to_csv("data/2018_social.csv")



In [97]:
coins_2021_df = []

In [98]:
persistent_scrapping(coins_2021_df, 200, '01-04-2021')

Processing bitcoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-04-2021localization=False
At index: 0
Processing ethereum
Requesting from URL: https://api.coingecko.com/api/v3/coins/ethereum/history?date=01-04-2021localization=False
At index: 1
Processing binancecoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/binancecoin/history?date=01-04-2021localization=False
At index: 2
Processing ripple
Requesting from URL: https://api.coingecko.com/api/v3/coins/ripple/history?date=01-04-2021localization=False
At index: 3
Processing tether
Requesting from URL: https://api.coingecko.com/api/v3/coins/tether/history?date=01-04-2021localization=False
At index: 4
Processing cardano
Requesting from URL: https://api.coingecko.com/api/v3/coins/cardano/history?date=01-04-2021localization=False
At index: 5
Processing dogecoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/dogecoin/history?date=01-04-2021localization=False
At index: 6
Proce

Processing hedera-hashgraph
Requesting from URL: https://api.coingecko.com/api/v3/coins/hedera-hashgraph/history?date=01-04-2021localization=False
At index: 57
Processing zilliqa
Requesting from URL: https://api.coingecko.com/api/v3/coins/zilliqa/history?date=01-04-2021localization=False
At index: 58
Processing celsius-degree-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/celsius-degree-token/history?date=01-04-2021localization=False
At index: 59
Processing enjincoin
Requesting from URL: https://api.coingecko.com/api/v3/coins/enjincoin/history?date=01-04-2021localization=False
At index: 60
Processing blockstack
Requesting from URL: https://api.coingecko.com/api/v3/coins/blockstack/history?date=01-04-2021localization=False
At index: 61
Processing leo-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/leo-token/history?date=01-04-2021localization=False
At index: 62
Processing amp-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/amp-token

Processing fantom
Requesting from URL: https://api.coingecko.com/api/v3/coins/fantom/history?date=01-04-2021localization=False
At index: 93
Processing xdce-crowd-sale
Requesting from URL: https://api.coingecko.com/api/v3/coins/xdce-crowd-sale/history?date=01-04-2021localization=False
At index: 94
Processing reserve-rights-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/reserve-rights-token/history?date=01-04-2021localization=False
At index: 95
Processing ecomi
Requesting from URL: https://api.coingecko.com/api/v3/coins/ecomi/history?date=01-04-2021localization=False
At index: 96
Processing iostoken
Requesting from URL: https://api.coingecko.com/api/v3/coins/iostoken/history?date=01-04-2021localization=False
At index: 97
Processing dent
Requesting from URL: https://api.coingecko.com/api/v3/coins/dent/history?date=01-04-2021localization=False
At index: 98
Processing paxos-standard
Requesting from URL: https://api.coingecko.com/api/v3/coins/paxos-standard/history?date=01

Counting... 27
Counting... 28
Counting... 29
Counting... 30
Processing bounce-token
You have reached 2 key errors and 2 timeouts.
Last data pulled at position index:131, id: bounce-token
Done with Data Retieval
We found a total of 129 entries
Processing mirror-protocol
Requesting from URL: https://api.coingecko.com/api/v3/coins/mirror-protocol/history?date=01-04-2021localization=False
At index: 129
Processing balancer
Requesting from URL: https://api.coingecko.com/api/v3/coins/balancer/history?date=01-04-2021localization=False
At index: 130
Processing bounce-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/bounce-token/history?date=01-04-2021localization=False
At index: 131
Processing gatechain-token
Requesting from URL: https://api.coingecko.com/api/v3/coins/gatechain-token/history?date=01-04-2021localization=False
At index: 132
Processing celo
Requesting from URL: https://api.coingecko.com/api/v3/coins/celo/history?date=01-04-2021localization=False
At index: 133
Proc

Processing orion-protocol
Requesting from URL: https://api.coingecko.com/api/v3/coins/orion-protocol/history?date=01-04-2021localization=False
At index: 180
Processing true-usd
Requesting from URL: https://api.coingecko.com/api/v3/coins/true-usd/history?date=01-04-2021localization=False
At index: 181
Processing swipe
Requesting from URL: https://api.coingecko.com/api/v3/coins/swipe/history?date=01-04-2021localization=False
At index: 182
Processing bakerytoken
Requesting from URL: https://api.coingecko.com/api/v3/coins/bakerytoken/history?date=01-04-2021localization=False
At index: 183
Processing ark
Requesting from URL: https://api.coingecko.com/api/v3/coins/ark/history?date=01-04-2021localization=False
At index: 184
Processing celer-network
Requesting from URL: https://api.coingecko.com/api/v3/coins/celer-network/history?date=01-04-2021localization=False
At index: 185
Processing rocket-pool
Requesting from URL: https://api.coingecko.com/api/v3/coins/rocket-pool/history?date=01-04-2021

In [99]:
coins_2021_df = pd.DataFrame(coins_2021_df)
coins_2021_df['market_cap'] = coins_2021_df['market_cap'] / 1000000
coins_2021_df.rename(columns={
    'market_cap' : 'market_cap_2021(millions)'
},inplace=True)
coins_2021_df


,id,price 01-04-2021,market_cap_2021(millions),reddit_subscribers 01-04-2021,twitter_followers 01-04-2021,alexa_rank 01-04-2021
0,bitcoin,58817.373888,1.098127e+06,2648989.0,83714.0,5018.0
1,ethereum,1915.832536,2.212107e+05,753614.0,852223.0,6152.0
2,binancecoin,302.745193,4.684157e+04,215806.0,NaN,154.0
3,ripple,0.573818,2.634585e+04,278986.0,1310270.0,11412.0
4,tether,1.000288,4.081691e+04,NaN,80505.0,38477.0
...,...,...,...,...,...,...
200,anchor-protocol,4.648715,2.198307e+02,NaN,NaN,53718.0
201,klever,0.087603,2.940475e+02,NaN,158334.0,59129.0
202,velas,0.161867,2.221146e+02,93.0,18798.0,168994.0
203,trustswap,4.428006,2.875174e+02,NaN,37260.0,26668.0


In [109]:
coins_2021_df.to_csv("data/2021_social.csv")

In [100]:
coins_2018_stats = pd.DataFrame(coins_2018_df.describe())
coins_2018_stats

,price 01-04-2018,market_cap_2018(millions),reddit_subscribers 01-04-2018,twitter_followers 01-04-2018,alexa_rank 01-04-2018
count,104.000000,104.000000,92.000000,103.000000,1.010000e+02
mean,95.100871,2399.536375,33098.945652,99108.737864,1.252211e+05
std,688.380391,12305.968917,93679.675577,136612.392029,1.989658e+05
min,0.000867,0.000000,7.000000,160.000000,2.300000e+02
25%,0.115632,68.516749,2859.250000,20587.500000,3.613700e+04
50%,0.755630,189.210337,6814.500000,56561.000000,8.189200e+04
75%,6.308137,601.121020,20366.000000,110823.500000,1.223660e+05
max,6975.275314,118234.055447,785619.000000,843082.000000,1.476215e+06


In [101]:
coins_2021_stats = pd.DataFrame(coins_2021_df.describe())
coins_2021_stats

,price 01-04-2021,market_cap_2021(millions),reddit_subscribers 01-04-2021,twitter_followers 01-04-2021,alexa_rank 01-04-2021
count,201.000000,2.010000e+02,1.300000e+02,1.730000e+02,1.970000e+02
mean,1437.322981,9.247779e+03,6.710914e+04,1.177146e+05,1.500329e+05
std,8617.567475,7.895844e+04,2.674787e+05,1.615271e+05,2.466216e+05
min,0.000294,0.000000e+00,9.300000e+01,4.000000e+01,1.540000e+02
25%,0.407197,4.013080e+02,3.407250e+03,2.894200e+04,2.628100e+04
50%,2.713329,8.142892e+02,8.854500e+03,6.921300e+04,6.321900e+04
75%,17.646010,2.301335e+03,2.811975e+04,1.283910e+05,1.424600e+05
max,59657.003140,1.098127e+06,2.648989e+06,1.310270e+06,1.640689e+06


In [102]:
# merge both data frames on the index
merged_stats = pd.merge(coins_2018_stats, coins_2021_stats, left_index=True, right_index=True)
merged_stats

,price 01-04-2018,market_cap_2018(millions),reddit_subscribers 01-04-2018,twitter_followers 01-04-2018,alexa_rank 01-04-2018,price 01-04-2021,market_cap_2021(millions),reddit_subscribers 01-04-2021,twitter_followers 01-04-2021,alexa_rank 01-04-2021
count,104.000000,104.000000,92.000000,103.000000,1.010000e+02,201.000000,2.010000e+02,1.300000e+02,1.730000e+02,1.970000e+02
mean,95.100871,2399.536375,33098.945652,99108.737864,1.252211e+05,1437.322981,9.247779e+03,6.710914e+04,1.177146e+05,1.500329e+05
std,688.380391,12305.968917,93679.675577,136612.392029,1.989658e+05,8617.567475,7.895844e+04,2.674787e+05,1.615271e+05,2.466216e+05
min,0.000867,0.000000,7.000000,160.000000,2.300000e+02,0.000294,0.000000e+00,9.300000e+01,4.000000e+01,1.540000e+02
25%,0.115632,68.516749,2859.250000,20587.500000,3.613700e+04,0.407197,4.013080e+02,3.407250e+03,2.894200e+04,2.628100e+04
50%,0.755630,189.210337,6814.500000,56561.000000,8.189200e+04,2.713329,8.142892e+02,8.854500e+03,6.921300e+04,6.321900e+04
75%,6.308137,601.121020,20366.000000,110823.500000,1.223660e+05,17.646010,2.301335e+03,2.811975e+04,1.283910e+05,1.424600e+05
max,6975.275314,118234.055447,785619.000000,843082.000000,1.476215e+06,59657.003140,1.098127e+06,2.648989e+06,1.310270e+06,1.640689e+06


In [105]:
# Add columns to calculate differences in yeard
merged_stats['price_difference'] = merged_stats['price 01-04-2021'] - merged_stats['price 01-04-2018']
merged_stats['market_cap_difference'] = merged_stats['market_cap_2021(millions)'] - merged_stats['market_cap_2018(millions)']
merged_stats['reddit_difference'] = merged_stats['reddit_subscribers 01-04-2021'] - merged_stats['reddit_subscribers 01-04-2018']
merged_stats['twitter_difference'] = merged_stats['twitter_followers 01-04-2021'] - merged_stats['twitter_followers 01-04-2018']
merged_stats['alexa_difference'] = merged_stats['alexa_rank 01-04-2021'] - merged_stats['alexa_rank 01-04-2018']

merged_stats

,price 01-04-2018,market_cap_2018(millions),reddit_subscribers 01-04-2018,twitter_followers 01-04-2018,alexa_rank 01-04-2018,price 01-04-2021,market_cap_2021(millions),reddit_subscribers 01-04-2021,twitter_followers 01-04-2021,alexa_rank 01-04-2021,price_difference,market_cap_difference,reddit_difference,twitter_difference,alexa_difference
count,104.000000,104.000000,92.000000,103.000000,1.010000e+02,201.000000,2.010000e+02,1.300000e+02,1.730000e+02,1.970000e+02,97.000000,97.000000,3.800000e+01,70.000000,96.000000
mean,95.100871,2399.536375,33098.945652,99108.737864,1.252211e+05,1437.322981,9.247779e+03,6.710914e+04,1.177146e+05,1.500329e+05,1342.222110,6848.242171,3.401019e+04,18605.869072,24811.714429
std,688.380391,12305.968917,93679.675577,136612.392029,1.989658e+05,8617.567475,7.895844e+04,2.674787e+05,1.615271e+05,2.466216e+05,7929.187084,66652.472506,1.737990e+05,24914.747731,47655.766999
min,0.000867,0.000000,7.000000,160.000000,2.300000e+02,0.000294,0.000000e+00,9.300000e+01,4.000000e+01,1.540000e+02,-0.000572,0.000000,8.600000e+01,-120.000000,-76.000000
25%,0.115632,68.516749,2859.250000,20587.500000,3.613700e+04,0.407197,4.013080e+02,3.407250e+03,2.894200e+04,2.628100e+04,0.291566,332.791249,5.480000e+02,8354.500000,-9856.000000
50%,0.755630,189.210337,6814.500000,56561.000000,8.189200e+04,2.713329,8.142892e+02,8.854500e+03,6.921300e+04,6.321900e+04,1.957699,625.078876,2.040000e+03,12652.000000,-18673.000000
75%,6.308137,601.121020,20366.000000,110823.500000,1.223660e+05,17.646010,2.301335e+03,2.811975e+04,1.283910e+05,1.424600e+05,11.337873,1700.214415,7.753750e+03,17567.500000,20094.000000
max,6975.275314,118234.055447,785619.000000,843082.000000,1.476215e+06,59657.003140,1.098127e+06,2.648989e+06,1.310270e+06,1.640689e+06,52681.727826,979893.255500,1.863370e+06,467188.000000,164474.000000


In [106]:
difference_stats = merged_stats[['price_difference', 'market_cap_difference','reddit_difference','twitter_difference','alexa_difference']]
difference_stats

,price_difference,market_cap_difference,reddit_difference,twitter_difference,alexa_difference
count,97.000000,97.000000,3.800000e+01,70.000000,96.000000
mean,1342.222110,6848.242171,3.401019e+04,18605.869072,24811.714429
std,7929.187084,66652.472506,1.737990e+05,24914.747731,47655.766999
min,-0.000572,0.000000,8.600000e+01,-120.000000,-76.000000
25%,0.291566,332.791249,5.480000e+02,8354.500000,-9856.000000
50%,1.957699,625.078876,2.040000e+03,12652.000000,-18673.000000
75%,11.337873,1700.214415,7.753750e+03,17567.500000,20094.000000
max,52681.727826,979893.255500,1.863370e+06,467188.000000,164474.000000
